In [1]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Minnesota Coronary Experiment

In [2]:
summary = Table(['Age', 'Condition', 'Total', 'Deaths', 'CHD Deaths']).with_rows([
    ['0-34',  'Diet',    1367, 3, 0],
    ['35-44', 'Diet',    728, 3, 0],
    ['45-54', 'Diet',    767, 14, 4],
    ['55-64', 'Diet',    870, 35, 7],
    ['65+',   'Diet',    953, 190, 42],
    ['0-34',  'Control', 1337, 7, 1],
    ['35-44', 'Control', 731, 4, 1],
    ['45-54', 'Control', 816, 16, 4],
    ['55-64', 'Control', 896, 33, 12],
    ['65+',   'Control', 958, 162, 34],   
])
summary

Age,Condition,Total,Deaths,CHD Deaths
0-34,Diet,1367,3,0
35-44,Diet,728,3,0
45-54,Diet,767,14,4
55-64,Diet,870,35,7
65+,Diet,953,190,42
0-34,Control,1337,7,1
35-44,Control,731,4,1
45-54,Control,816,16,4
55-64,Control,896,33,12
65+,Control,958,162,34


In [4]:
subjects = Table(['Age', 'Condition', 'Participated', 'Died'])
for row in summary.rows:
    i = np.arange(0, row.item('Total'))
    t = Table().with_columns('Died', i < row.item('Deaths'))
    t.append_column('Age', row.item('Age'))
    t.append_column('Condition', row.item('Condition'))
    t.append_column('Participated', True)
    subjects.append(t)

subjects.sample(10)

Age,Condition,Participated,Died
55-64,Control,True,False
55-64,Diet,True,False
55-64,Diet,True,False
0-34,Diet,True,False
55-64,Control,True,False
35-44,Control,True,False
45-54,Diet,True,False
65+,Control,True,False
55-64,Control,True,False
65+,Diet,True,False


In [5]:
subjects.group(['Age', 'Condition'], sum)

Age,Condition,Participated sum,Died sum
0-34,Control,1337,7
0-34,Diet,1367,3
35-44,Control,731,4
35-44,Diet,728,3
45-54,Control,816,16
45-54,Diet,767,14
55-64,Control,896,33
55-64,Diet,870,35
65+,Control,958,162
65+,Diet,953,190


In [16]:
oldest = subjects.where('Age', '65+')
oldest.group('Condition', sum)

Condition,Age sum,Participated sum,Died sum
Control,,958,162
Diet,,953,190


In [17]:
(190/953) - (162/958)  # the death (hazard) rate was a bit *higher* for the diet group!

0.030268112783058437

In [ ]:
# Null hypothesis: the difference in hazard rates was the same for the control group and diet group
# Simulated value: absolute difference in hazard rate for control/diet

In [18]:
def hazard_rate(counts):
    return counts.item('Died sum') / counts.item('Participated sum')

def rate_difference(t):
    counts = t.drop('Age').group('Condition', sum)
    return abs(hazard_rate(counts.row(1)) - hazard_rate(counts.row(0)))

rate_difference(subjects) # barely any difference for the entire population

0.0054393439270044933

In [19]:
rate_difference(subjects.where('Age', '0-34'))

0.0030410154080667343

In [20]:
rate_difference(subjects.where('Age', '65+'))

0.030268112783058437

In [23]:
def test(t):
    observed = rate_difference(t)
    repetitions = 200

    stats = make_array()
    for i in np.arange(repetitions):
        # permutation test: shuffle who died and check the rate difference
        simulated_results = t.select('Died').sample().column('Died')
        simulated_outcomes = t.with_column('Died', simulated_results)
        simulated_stat = rate_difference(simulated_outcomes)
        stats = np.append(stats, simulated_stat)

    # Find the empirical P-value:
    p = np.count_nonzero(stats >= observed) / repetitions
    
    print('  Observed absolute difference in hazard rates:', observed)
    print('  P-value:', p)

test(subjects)

  Observed absolute difference in hazard rates: 0.005439343927
  P-value: 0.19


In [24]:
for age in subjects.group('Age').column('Age'):
    print('Ages', age)
    test(subjects.where('Age', age))

Ages 0-34
  Observed absolute difference in hazard rates: 0.00304101540807
  P-value: 0.18
Ages 35-44
  Observed absolute difference in hazard rates: 0.00135107710347
  P-value: 0.785
Ages 45-54
  Observed absolute difference in hazard rates: 0.00135490963008
  P-value: 0.855
Ages 55-64
  Observed absolute difference in hazard rates: 0.00339952791461
  P-value: 0.675
Ages 65+
  Observed absolute difference in hazard rates: 0.0302681127831
  P-value: 0.07


# Estimating Exam Scores

In [28]:
scores = Table.read_table('data/scores.csv')
scores

Midterm 1,Midterm 2,Mentored
28,20,False
28.5,35,False
23.5,13.5,False
24.5,22.5,True
28.5,35.5,True
22.5,28,True
24,22.5,False
29,36,True
19.5,20,False
19.5,20,True


In [29]:
scores.drop(2) # let's forget about column 2

Midterm 1,Midterm 2
28,20
28.5,35
23.5,13.5
24.5,22.5
28.5,35.5
22.5,28
24,22.5
29,36
19.5,20
19.5,20
